In [ ]:
import sys
import torch
import numpy as np

# Add your path of library
sys.path.append('../gw_code_submission')
from model import Model
from binary_classifier import SimpleBinaryClassifier
from download_model_weight import download_model_weight

sys.path.append('../gw_train')
sys.path.append('../gw_train/Datasets')
from model_train import train_classification

In [ ]:
#########################################
#      Training model
#########################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

in_dim = 100*2
model = SimpleBinaryClassifier(in_dim, hidden_dim=128).to(device)

# example datasets (First one should be background related data the others should be datas with signal)
datasets = ("background_train.npz", "bbh_for_challenge_train.npy", "sglf_for_challenge_train.npy")

custom_args = {'lr': 1e-3, 'num_epochs': 200, 'batch_size': 100000}  

train_classification(
    model,
    datasets,  
    "model_name.pth",
    device,
    epochs=custom_args['num_epochs'],
    batch_size=custom_args['batch_size'],
    lr_base=custom_args['lr']
)

In [ ]:
###########################################
#        Using model for prediction
###########################################

# Clean up variable {model}
del model

# Create Model instance
model = Model()

# Download model weight
download_model_weight()

# Load trained model
model.load("supervised_model.pth")

# Before running the code, please first download "sglf_for_challenge.npy" and place it in under "../gw_train/Datasets/"
# Make prediction based on {model} on {device} using data at {predict_data_dir} and save the prediction to {csv_path}
X_test = np.load("../gw_train/Datasets/sglf_for_challenge.npy") # shape=(N, 2, 200), which should be test data(X_test/stds)
stds = np.std(X_test, axis=-1)[:, :, np.newaxis]
X_test = X_test/stds
X_test = np.swapaxes(X_test, 1, 2)

model.predict(X_test[:10])